# PPT Generator

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

C:\Users\imbit\AppData\Local\Temp\ipykernel_6964\2314480482.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
load_dotenv()

True

In [3]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
import google.generativeai as genai

In [5]:
genai.configure(api_key=google_api_key)

In [6]:
model = genai.GenerativeModel('gemini-pro')

In [59]:
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#### List All The DOCX Files


First We Get All the DocX Files from the folder

In [8]:
folder = "./Data/"

In [9]:
import glob

In [10]:
def list_docx_files(folder_path):
    doc_files = glob.glob(os.path.join(folder_path,"*.docx"))
    return doc_files

In [11]:
files = list_docx_files(folder)

In [52]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

In [62]:
loader  = Docx2txtLoader(files[0])
data = loader.load()

text_splitter =  RecursiveCharacterTextSplitter(chunk_size =10000,chunk_overlap = 1000)
docs = text_splitter.split_documents(data)

model = 'models/embedding-001'

embedding = GoogleGenerativeAIEmbeddings(
    model = model, 
    task_type='retrieval_document'
)
db =Chroma.from_documents(docs,embedding= embedding,persist_directory="./Data/tempchroma_db")


In [13]:
def extract_content_from_files(folder_path):
    frame = []
    doc_files = glob.glob(os.path.join(folder_path,"*.docx"))
    for file in doc_files:
        loader  = Docx2txtLoader(file)
        data = loader.load()

        text_splitter =  RecursiveCharacterTextSplitter(chunk_size =10000,chunk_overlap = 1000)
        docs = text_splitter.split_documents(data)
        # content = ""
        # for para in document.paragraphs:
        #     content += para.text + "\n"
        embeddings = process_in_batches(content,1000) 
        frame.append({'File Name':file,'Content':content,'Embeddings':embeddings})
    return pd.DataFrame(frame)

In [14]:
def create_embeddings(text):
    model = 'models/embedding-001'
    return genai.embed_content(
        model = model,
        content=text,
        task_type='retrieval_document',
    )["embedding"]
    
    

In [15]:
#process in batches

def process_in_batches(chunks, batch_size):
    start_index = 0
    embeddings = []
    while start_index < len(chunks):
        end_index = start_index + batch_size
        if start_index == 0:
            data = chunks[start_index:end_index]
            embeddings.append(create_embeddings(data))
        else:
            if end_index > len(chunks):
                data = chunks[start_index:len(chunks)]
            else:
                data = chunks[start_index:end_index]
            embeddings.append(create_embeddings(data))
        start_index = end_index
        return embeddings


In [16]:
data =extract_content_from_files(folder)

<module 'langchain_community.vectorstores.chroma' from 'C:\\Users\\imbit\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\langchain_community\\vectorstores\\chroma.py'>

Asking the Query and Checking the similarities

In [61]:
query = "What Did PUMA do in India?"
docs = db.similarity_search(query)

docs[0].page_content

"OVERVIEW 2022\n\nNotes relating to forward-looking statements\n\nThis document contains statements about the future business development and strategic direction of the Company. The forward-looking statements are based on management's current expectations and assumptions. They are subject to certain risks and fluctuations as described in other publications, in particular in the risk and opportunities management section of the combined management report. If these expectations and assumptions do not apply or if unforeseen risks arise, the actual course of business may differ significantly from the expected developments. We therefore assume no liability for the accuracy of these forecasts.\n\n\n\n\n\n\n\nThese sections contain content or cross-references not required by law, which were not audited by the auditor, but were merely read critically. In the case of cross- references, the information to which the cross- references refer was also not audited.\n\n\n\n\n\n\n\n\n\n➔HUBERT HINTERSEH

In [19]:
def get_similar_answer(query,dataframe):
    query_embedding = genai.embed_content(
        model = model,
        content=query,
        task_type='retrieval_query',
    )
    # Using dot product of stacked/accu,lated dataframe embeddings and query embedding
    dot_products = np.dot(np.stack(dataframe['Embeddings']),query_embedding["embedding"])
    idxs = np.argmax(dot_products)
    
    
    

In [20]:
 dot_products = np.dot(np.stack(data['Embeddings']),request["embedding"])

In [21]:
dot_products

array([[0.66765165]])

In [25]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("C:/Users/imbit/Downloads/Overview 2022 _ Puma AR 2022.pdf")
loader.load()

ModuleNotFoundError: No module named 'pwd'